# Orchestrator Agent Tutorial

This tutorial demonstrates how to create an orchestrator agent using the Atomic Agents library. The orchestrator agent will manage tasks and respond to user inputs in rhyming verse.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KennyVaneetvelde/atomic_agents/blob/main/examples/notebooks/orchestrator.ipynb#)


# Prerequisites

Before proceeding with this notebook, it is highly recommended to read up on the basics of the following libraries:

- **Pydantic**: A data validation and settings management library using Python type annotations. You can find more information and documentation at [Pydantic GitHub](https://github.com/pydantic/pydantic).
- **Instructor**: A Python library that simplifies working with structured outputs from large language models (LLMs). It provides a user-friendly API to manage validation, retries, and streaming responses. More details can be found at [Instructor GitHub](https://github.com/jxnl/instructor).

Understanding these libraries will help you make the most of this library.


# Install Necessary Packages

First, we need to install the required packages. Run the following command to install `atomic-agents`, `openai`, and `instructor` libraries.

In [ ]:
# Install necessary packages
%pip install atomic-agents openai instructor

# Import Libraries

We will import the necessary libraries for creating the orchestrator agent. Each library serves a specific purpose:
- `ChatMemory`: Manages the chat history.
- `BaseChatAgent`: The base class to create a custom chatbot. Can be extended for additional functionality if needed.
- `SystemPromptGenerator` and `SystemPromptInfo`: To define and generate system prompts.
- `BaseTool`, `CalculatorTool`, `SearxNGSearchTool`: Tools that the orchestrator agent can use to perform tasks.

In [ ]:
import os
from typing import List, Type, Union
from pydantic import BaseModel, Field
from rich.console import Console
from rich.markdown import Markdown
import instructor
import openai
from atomic_agents.lib.components.chat_memory import ChatMemory
from atomic_agents.agents.base_chat_agent import BaseChatAgent, BaseChatAgentResponse, BaseChatAgentConfig
from atomic_agents.lib.components.system_prompt_generator import SystemPromptContextProviderBase, SystemPromptGenerator, SystemPromptInfo
from atomic_agents.lib.tools.base import BaseTool
from atomic_agents.lib.tools.calculator_tool import CalculatorTool, CalculatorToolSchema
from atomic_agents.lib.tools.search.searx_tool import SearxNGSearchTool, SearxNGSearchToolConfig, SearxNGSearchToolSchema

console = Console()


# Set Up API Keys

To use the **OpenAI** and **SearxNG** APIs, you need to set up your API keys. You can either enter them directly in the code or set them as environment variables.

In [ ]:
##################################################################
# ENTER YOUR OPENAI API KEY BELOW, OR SET IT AS AN ENVIRONMENT VARIABLE #
##################################################################
API_KEY = ''
if not API_KEY:
    # Get the environment variable
    API_KEY = os.getenv('OPENAI_API_KEY')

if not API_KEY:
    raise ValueError('API key is not set. Please set the API key as a static variable or in the environment variable OPENAI_API_KEY.')

client = instructor.from_openai(openai.OpenAI(api_key=API_KEY))


# Define System Prompt Information

In this step, we will define the system prompt information including background, steps, and output instructions. This helps the orchestrator agent understand how to respond to user inputs and manage tasks.

### Tool Definition

First, we define the tools that the orchestrator agent can use. These tools will help the agent perform various tasks such as searching the web or performing calculations.

In [ ]:
# Initialize tools
search_tool = SearxNGSearchTool(SearxNGSearchToolConfig(base_url=os.getenv('SEARXNG_BASE_URL'), max_results=10))
calculator_tool = CalculatorTool()

Now that we have defined the tools, we can define the ToolInfoProvider. This will format the tool information in a way that is nice to put in the system prompt.

In [ ]:
class ToolInfoProvider(SystemPromptContextProviderBase):        
    def get_info(self) -> str:
        response = 'The available tools are:\n'
        for tool in [search_tool, calculator_tool]:
            response += f'- {tool.tool_name}: {tool.tool_description}\n'
        return response

## System Prompt Generator

Next, we define the system prompt information including background, steps, output instructions and the ToolInfoProvider we defined above. This helps the orchestrator agent understand how to respond to user inputs and manage tasks.

In [ ]:
# Define system prompt information including background, steps, and output instructions
system_prompt = SystemPromptInfo(
    background=[
        'This assistant is an orchestrator of tasks designed to be helpful and efficient.',
    ],
    steps=[
        'Understand the user\'s input and the current context.',
        'Take a step back and think methodically and step-by-step about how to proceed by using internal reasoning.',
        'Evaluate the available tools and decide whether to use a tool based on the current context, user input, and history.',
        'If a tool can be used, decide which tool to use and how to use it.',
        'If a tool can be used it must always be explicitly mentioned in the internal reasoning response.',
        'If a tool can be used, return nothing but the tool response to the user. If no tool is needed or if the tool has finished, return a chat response.',
        'Execute the chosen tool and provide a relevant response to the user.'
    ],
    output_instructions=[
        'Provide helpful and relevant information to assist the user.',
        'Be efficient and effective in task orchestration.',
        'Always answer in rhyming verse.'
    ],
    context_providers={
        'tools': ToolInfoProvider(title='Available tools')
    }
)

# Initialize the system prompt generator with the defined system prompt and dynamic info providers
system_prompt_generator = SystemPromptGenerator(system_prompt)

# Test out the system prompt generator
from rich.markdown import Markdown
from rich.console import Console
console.print(Markdown(system_prompt_generator.generate_prompt()))

## Initialize Chat Memory

We will initialize the chat memory to store conversation history and load an initial greeting message.

In [ ]:
memory = ChatMemory()
initial_memory = [
    {'role': 'assistant', 'content': 'How do you do? What can I do for you? Tell me, pray, what is your need today?'}
]
memory.load(initial_memory)

## Define InternalReasoningResponse
To improve the agent's responses, we define an InternalReasoningResponse class that will be interspersed with the agent's responses. This class will help the agent reason about the next steps in the conversation.

In [ ]:
# Define InternalReasoningResponse class
class InternalReasoningResponse(BaseModel):
    observation: str = Field(..., description='What is the current state of the conversation and context? What is the user saying or asking?')
    action_plan: List[str] = Field(..., min_length=1, description='What steps could be taken to address the current observation? Is there a tool that could be used?')

    class Config:
        title = 'InternalReasoningResponse'
        description = 'The internal reasoning response schema for the chat agent, following the ReACT pattern.'
        json_schema_extra = {
            'title': title,
            'description': description,
        }

# Define ResponseSchema class
class ResponseSchema(BaseModel):
    chosen_schema: Union[BaseChatAgentResponse, SearxNGSearchToolSchema, CalculatorToolSchema] = Field(..., description='The response from the chat agent, which may include the result of using a tool if one was deemed necessary.')
    class Config:
        title = 'ResponseSchema'
        description = 'The response schema for the chat agent, including potential tool usage results.'
        json_schema_extra = {
            'title': title,
            'description': description,
        }


Now that we have defined the necessary components, we can create the orchestrator agent.
Personally, I like to extend the OrchestratorAgentConfig even if I don't add any additional fields. This way, I can easily add additional fields in the future if needed with minimal impact on the existing code.

In [ ]:

# Define OrchestratorAgentConfig class
class OrchestratorAgentConfig(BaseChatAgentConfig):
    pass

# Define OrchestratorAgent class
class OrchestratorAgent(BaseChatAgent):
    def _pre_run(self):
        self.memory.add_message('assistant', 'First, I will plan my steps and think about the tools at my disposal.')
        response = self.get_response(response_model=InternalReasoningResponse)
        self.memory.add_message('assistant', f'INTERNAL THOUGHT: I have observed "{response.observation}" and will take the following steps: {", ".join(response.action_plan)}')
        return


Now that everything is set up, we can create an instance of the orchestrator agent and start interacting with it. Try asking it questions such as _"What is 10 + log(5)?"_ or _"Can you find information about the history of the internet?"_ and see how it responds.

To keep this guide simple and to the point, we don't actually call the tools and do further processing, however since everything is typed you can check the type of the response and call the appropriate tool.

In [ ]:

# Create an instance of OrchestratorAgent with the specified configuration
agent = OrchestratorAgent(
    config=OrchestratorAgentConfig(
        client=instructor.from_openai(openai.OpenAI()),
        model='gpt-3.5-turbo',
        system_prompt_generator=system_prompt_generator,
        memory=memory,
        output_schema=ResponseSchema
    )
)

# Print the initial message from the assistant
console.print(f'Agent: {initial_memory[0]["content"]}')

while True:
    user_input = input('You: ')
    if user_input.lower() in ['/exit', '/quit']:
        print('Exiting chat...')
        break

    response = agent.run(agent.input_schema(chat_input=user_input))
    console.print(f'Agent: {response}')
